In [45]:
import pandas as pd

### Choisir le match

In [46]:
rep="/Users/gilles/Github/AMBK-SportsCom/Chaines/Traitements_GB/"
match="FrNz"
# match="FrAr"
nFile=match+"-RefSynTour.xlsx"

writer = pd.ExcelWriter(rep+"Matches-Stats-RefSynTour.xlsx", engine="openpyxl",mode="a")

In [47]:
df=pd.read_excel(rep+nFile,index_col=0)
df["type"]=df["type"].str.replace("Color","Colour")
df.head()

,Transcription,Cat,Lemme,Referent,numChaine,chaineIndex,chaineDep,chaineApp,nPhrase,tour,type,tDeb,tFin,speaker,Mi-temps
0,bonsoir,INT,bonsoir,NaN,NaN,NaN,NaN,NaN,0,1-spk1,Colour commentary,00:00:36.628000,00:00:40.304000,spk1,1.0
1,",",PUN,",",NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,1.0
2,c',PRO:DEM,ce,NaN,NaN,NaN,NaN,NaN,0,1-spk1,Colour commentary,00:00:36.628000,00:00:40.304000,spk1,1.0
3,était,VER:impf,être,NaN,NaN,NaN,NaN,NaN,0,1-spk1,Colour commentary,00:00:36.628000,00:00:40.304000,spk1,1.0
4,ceux,PRO:DEM,celui,GTF,1.0,1.0,0.0,NaN,0,1-spk1,Colour commentary,00:00:36.628000,00:00:40.304000,spk1,1.0


In [48]:
nbMots=len(df[~df.Cat.isin(["PUN","SENT"])])
print(match,nbMots)

FrNz 18211


## Distribution globale des longueurs de chaines

In [49]:
maxNumChaine=int(df.numChaine.max())
maxNumChaine

1150

### Extraire les chaines par mi-temps

In [50]:
def extractChaines(df,mt=[1,2]):
    dChaines={}
    lChaines={}
    for num in range(maxNumChaine):
        n=num+1
        dfChaine=df.loc[df.numChaine==n]
        if len(dfChaine)>0 and dfChaine.iloc[0]["Mi-temps"] in mt:
            try:
                dChaines[n]={"ref":dfChaine.Referent.values[0],"chaine":{}}
                indexMax=int(dfChaine.chaineIndex.max())
                if indexMax not in lChaines:
                    lChaines[indexMax]=0
                lChaines[indexMax]+=1
                for ind in range(indexMax):
                    i=ind+1
                    dfIndex=dfChaine.loc[dfChaine.chaineIndex==i]
                    dChaines[n]["chaine"][i]=dfIndex[["Cat",
                                                     "Transcription",
                                                     "speaker",
                                                     "Mi-temps"]].to_dict()
            except:
                print("pb",n,end=", ")
                # display(dfChaine)
    return dChaines,lChaines

In [51]:
chaines,lChaines=extractChaines(df,mt=[1,2])
chainesM1,lChainesM1=extractChaines(df,mt=[1])
chainesM2,lChainesM2=extractChaines(df,mt=[2])

In [52]:
print(lChaines)
print(lChainesM1)
print(lChainesM2)

{2: 222, 1: 694, 3: 86, 10: 7, 5: 25, 4: 49, 6: 18, 7: 14, 8: 10, 11: 2, 9: 3, 14: 1}
{2: 112, 1: 363, 3: 44, 10: 4, 5: 13, 4: 24, 6: 7, 7: 6, 8: 5, 11: 1, 9: 1}
{1: 331, 2: 110, 10: 3, 5: 12, 3: 42, 4: 25, 11: 1, 6: 11, 8: 5, 7: 8, 9: 2, 14: 1}


### Assemblage de la distribution des longueurs
- par mi-temps *MT1*, *MT2*
- pour le match *Match*

In [53]:
sM1=pd.Series(lChainesM1).sort_index()
sM2=pd.Series(lChainesM2).sort_index()
dfDistLong=pd.DataFrame([sM1,sM2]).T.fillna(0)
dfDistLong.columns="MT1 MT2".split(" ")
dfDistLong["Match"]=dfDistLong.MT1+dfDistLong.MT2
dfDistLong.columns=[c+"-%s"%match for c in dfDistLong.columns]
maxLChaines=dfDistLong.index.max()
dfDistLong

,MT1-FrNz,MT2-FrNz,Match-FrNz
1,363.0,331.0,694.0
2,112.0,110.0,222.0
3,44.0,42.0,86.0
4,24.0,25.0,49.0
5,13.0,12.0,25.0
6,7.0,11.0,18.0
7,6.0,8.0,14.0
8,5.0,5.0,10.0
9,1.0,2.0,3.0
10,4.0,3.0,7.0


#### Nombre de chaines par longueur

In [54]:
dfDistLong.to_excel(writer, sheet_name="Dist-Long-%s"%match)

## Distribution des longueurs par référent

In [55]:
def extractRef(locChaines):
    refChaines={}
    lRefChaines={}#
    # print(lRefChaines)
    for k,v in locChaines.items():
        ref=locChaines[k]["ref"]
        chaine=locChaines[k]["chaine"]
        if ref not in refChaines:
            refChaines[ref]=[]
            lRefChaines[ref]={i:0 for i in range(1,maxLChaines+1)}
        refChaines[ref].append(locChaines[k]["chaine"])
        longueur=max(locChaines[k]["chaine"].keys())
        if longueur not in lRefChaines[ref]:
            lRefChaines[ref][longueur]=0
        lRefChaines[ref][longueur]+=1
    dfRefChaines=pd.DataFrame.from_dict(lRefChaines).sort_index().T
    return dfRefChaines

In [56]:
rM1=extractRef(chainesM1)
rM2=extractRef(chainesM2)

In [57]:
dfRefChaines=rM1.join(rM2, how='outer',lsuffix="-MT1",rsuffix="-MT2").fillna(0)
for c1 in dfRefChaines.columns[:maxLChaines]:
    c2=c1.replace("MT1","MT2")
    c=c1.replace("MT1","Match")
    dfRefChaines[c]=dfRefChaines[c1]+dfRefChaines[c2]
dfRefChaines["sum"]=dfRefChaines.sum(axis=1)
dfRefChaines=dfRefChaines.sort_values(by='sum',ascending=False)
dfRefChaines=dfRefChaines.drop(columns="sum")
dfRefChaines.columns=[c+"-%s"%match for c in dfRefChaines.columns]
dfRefChaines

,1-MT1-FrNz,2-MT1-FrNz,3-MT1-FrNz,4-MT1-FrNz,5-MT1-FrNz,6-MT1-FrNz,7-MT1-FrNz,8-MT1-FrNz,9-MT1-FrNz,10-MT1-FrNz,...,5-Match-FrNz,6-Match-FrNz,7-Match-FrNz,8-Match-FrNz,9-Match-FrNz,10-Match-FrNz,11-Match-FrNz,12-Match-FrNz,13-Match-FrNz,14-Match-FrNz
GTF,46.0,21.0,6.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,...,5.0,2.0,2.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0
GTNZ,40.0,11.0,7.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,...,3.0,3.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
CHRISTIAN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Bernard,22.0,2.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Christian,32.0,3.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,...,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ID,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
J-F7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
J-F20,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
J-F3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [58]:
dfRefChaines.to_excel(writer, sheet_name="Ref-Dist-Long-%s"%match)

#### Calcul des roots par catégorie
- chaineIndex renseignée
- chaineDep==0 root

In [59]:
lCat=df[(df.chaineIndex.notnull())&(df.chaineDep==0)]["Cat"].unique().tolist()
print(lCat)
lCat=[c for c in lCat if c in "NAM PRO:REL PRO:PER NOM DET:POS PRO:DEM PRO:IND PRP:det".split(" ")]
lCat

['PRO:DEM', 'NAM', 'DET:ART', 'NOM', 'PRP:det', 'PRP', 'NUM', 'ADV', 'VER:simp', 'ADJ', 'PRO:IND', 'PUN', 'PRO:PER', 'VER:infi', 'VER:pper', 'VER:pres', 'DET:POS', 'INT']


['PRO:DEM', 'NAM', 'NOM', 'PRP:det', 'PRO:IND', 'PRO:PER', 'DET:POS']

In [60]:
rootTableM1=df[(df.chaineIndex.notnull())&(df.chaineDep==0)&(df["Mi-temps"]==1)].groupby("Cat").count()[["Lemme"]].loc[lCat]
rootTableM2=df[(df.chaineIndex.notnull())&(df.chaineDep==0)&(df["Mi-temps"]==2)].groupby("Cat").count()[["Lemme"]].loc[lCat]

In [61]:
rootTable=rootTableM1.join(rootTableM2, how='outer',lsuffix="M1",rsuffix="M2").fillna(0)
rootTable.columns=["MT1","MT2"]
rootTable["Match"]=rootTable["MT1"]+rootTable["MT2"]
rootTable.columns=[c+"-%s"%match for c in rootTable.columns]
rootTable

,MT1-FrNz,MT2-FrNz,Match-FrNz
Cat,,,
PRO:DEM,5,5,10
NAM,533,510,1043
NOM,179,138,317
PRP:det,19,4,23
PRO:IND,1,1,2
PRO:PER,87,106,193
DET:POS,1,2,3


In [62]:
rootTable.to_excel(writer, sheet_name="Dist-Cat-root-%s"%match)

#### Calcul du nombre de reprises par catégorie
- chaineIndex renseignée
- chaineDep différent de root (≠0)

In [63]:
catTableM1=df[(df.chaineIndex.notnull())&(df.chaineDep>0)&(df["Mi-temps"]==1)].groupby("Cat").count()[["Lemme"]]#.loc[lCat]
catTableM2=df[(df.chaineIndex.notnull())&(df.chaineDep>0)&(df["Mi-temps"]==2)].groupby("Cat").count()[["Lemme"]]#.loc[lCat]

In [64]:
catTable=catTableM1.join(catTableM2, how='outer',lsuffix="M1",rsuffix="M2").fillna(0)
catTable.columns=["MT1","MT2"]
catTable["Match"]=catTable["MT1"]+catTable["MT2"]
catTable.columns=[c+"-%s"%match for c in catTable.columns]
catTable

,MT1-FrNz,MT2-FrNz,Match-FrNz
Cat,,,
ADJ,7.0,15.0,22.0
ADV,1.0,0.0,1.0
DET:ART,23.0,10.0,33.0
DET:POS,11.0,3.0,14.0
INT,1.0,0.0,1.0
KON,2.0,0.0,2.0
NAM,57.0,10.0,67.0
NOM,45.0,36.0,81.0
NUM,4.0,9.0,13.0


In [65]:
catTable.to_excel(writer, sheet_name="Dist-Cat-reprises-%s"%match)

#### Détail des reprises

In [66]:
dfGroupReprises=df[(df.chaineIndex.notnull())&(df.chaineDep>0)].groupby(["numChaine","chaineIndex"])[["Cat","Transcription"]].agg(lambda x: " ".join(x))
dfGroupReprises.to_excel(writer, sheet_name="Reprises-%s"%match)

In [67]:
print(match,len(dfGroupReprises),nbMots,"%2.1f%%"%(len(dfGroupReprises)/nbMots*100))

FrNz 865 18211 4.7%


#### Détail des références

In [68]:
dfGroupeReferents=df[(df.chaineIndex.notnull())].groupby(["numChaine","chaineIndex"])[["Cat","Transcription"]].agg(" ".join)
dfGroupeReferents.to_excel(writer, sheet_name="Ref-Groupes-%s"%match)
dfGroupeReferents

Cat  \
numChaine chaineIndex                                    
1.0       1.0                                  PRO:DEM   
          2.0                                  PRO:DEM   
2.0       1.0                                      NAM   
3.0       1.0                                      NAM   
4.0       1.0          DET:ART NOM PRP:det NOM PRP NOM   
...                                                ...   
1147.0    8.0              NOM PRP DET:ART NOM PRP NAM   
1148.0    1.0                                      NAM   
1149.0    1.0                      DET:ART NOM PRP NAM   
          2.0                                  PRO:PER   
1150.0    1.0                                      NAM   

                                          Transcription  
numChaine chaineIndex                                    
1.0       1.0                                      ceux  
          2.0                                      ceux  
2.0       1.0                                   Bernard  
3.0       1.0                                 Christian  
4.0       1.0           les champions du monde en titre  
...                                                 ...  
1147.0    8.0          capitaine de l' équipe de France  
1148.0    1.0                                   Bernard  
1149.0    1.0                       l' équipe de France  
          2.0                                      nous  
1150.0    1.0                                   Bernard  

[2050 rows x 2 columns]

Densité des références

In [69]:
print(match,len(dfGroupeReferents),nbMots,"%2.1f%%"%(len(dfGroupeReferents)/nbMots*100))

FrNz 2050 18211 11.3%


In [70]:
# print(dfGroupeReferents.to_string())

### Stats par speaker

In [71]:
def extractSpk(df,mt=[1,2]):
    chainesSpk={}
    spkChaines={}
    for num in range(maxNumChaine):
        n=num+1
        dfChaine=df.loc[df.numChaine==n]
        if len(dfChaine)>0 and dfChaine.iloc[0]["Mi-temps"] in mt:
            # display(dfChaine)
            try:
                chainesSpk[n]={"spk":dfChaine.speaker.values[0],
                               "ref":dfChaine.Referent.values[0],
                               "chaine":{}}
                indexMax=int(dfChaine.chaineIndex.max())
                # print(indexMax)
                if indexMax not in spkChaines:
                    spkChaines[indexMax]=0
                spkChaines[indexMax]+=1
                for ind in range(indexMax):
                    i=ind+1
                    dfIndex=dfChaine.loc[dfChaine.chaineIndex==i]
                    # display(dfIndex)
                    chainesSpk[n]["chaine"][i]=dfIndex[["Cat","Transcription"]].to_dict()
            except:
                print("pb",n,end=", ")
    return chainesSpk,spkChaines

In [72]:
chainesSpk,spkChaines=extractSpk(df,mt=[1,2])
chainesSpkM1,spkChainesM1=extractSpk(df,mt=[1])
chainesSpkM2,spkChainesM2=extractSpk(df,mt=[2])

In [73]:
print(spkChaines)
print(spkChainesM1)
print(spkChainesM2)


{2: 222, 1: 694, 3: 86, 10: 7, 5: 25, 4: 49, 6: 18, 7: 14, 8: 10, 11: 2, 9: 3, 14: 1}
{2: 112, 1: 363, 3: 44, 10: 4, 5: 13, 4: 24, 6: 7, 7: 6, 8: 5, 11: 1, 9: 1}
{1: 331, 2: 110, 10: 3, 5: 12, 3: 42, 4: 25, 11: 1, 6: 11, 8: 5, 7: 8, 9: 2, 14: 1}


#### longueur de chaines par speaker

In [74]:
def longChainesSpk(chainesSpk):
    spkLen={"spk1":{n+1:0 for n in range(14)},"spk2":{n+1:0 for n in range(14)}}
    
    for k,v in chainesSpk.items():
        if v['spk']==v['spk']:
            spks=v['spk'].split(" ")
            for spk in spks:
                lK=len(v["chaine"])
                if spk not in spkLen:
                    spkLen[spk]={}
                if lK not in spkLen[spk]:
                    spkLen[spk][lK]=0
                spkLen[spk][lK]+=1
    return spkLen

In [75]:
spkLen=longChainesSpk(chainesSpk)
spkLenM1=longChainesSpk(chainesSpkM1)
spkLenM2=longChainesSpk(chainesSpkM2)

In [76]:
dfSpkLongM1=pd.DataFrame.from_dict(spkLenM1,orient="index").sort_index().T[["spk1","spk2"]]
dfSpkLongM2=pd.DataFrame.from_dict(spkLenM2,orient="index").sort_index().T[["spk1","spk2"]]
dfSpkLong=pd.concat([dfSpkLongM1,dfSpkLongM2],axis=1)
dfSpkLong.columns=["spk1-MT1","spk2-MT1","spk1-MT2","spk2-MT2"]
dfSpkLong["spk1-Match"]=dfSpkLong["spk1-MT1"]+dfSpkLong["spk1-MT2"]
dfSpkLong["spk2-Match"]=dfSpkLong["spk2-MT1"]+dfSpkLong["spk2-MT2"]
dfSpkLong.columns=[c+"-%s"%match for c in dfSpkLong.columns]
dfSpkLong

,spk1-MT1-FrNz,spk2-MT1-FrNz,spk1-MT2-FrNz,spk2-MT2-FrNz,spk1-Match-FrNz,spk2-Match-FrNz
1,255.0,120.0,187.0,138.0,442.0,258.0
2,72.0,45.0,48.0,56.0,120.0,101.0
3,26.0,19.0,15.0,26.0,41.0,45.0
4,11.0,14.0,10.0,18.0,21.0,32.0
5,8.0,6.0,4.0,11.0,12.0,17.0
6,2.0,5.0,8.0,3.0,10.0,8.0
7,3.0,4.0,4.0,5.0,7.0,9.0
8,3.0,3.0,0.0,3.0,3.0,6.0
9,0.0,1.0,0.0,2.0,0.0,3.0
10,2.0,2.0,1.0,1.0,3.0,3.0


In [77]:
dfSpkLong.to_excel(writer, sheet_name="Spk-Dist-Long-%s"%match)

In [78]:
dfGroupeReferents=df[(df.chaineIndex.notnull())].groupby(["speaker","numChaine","chaineIndex"])[["Cat","Transcription"]].agg(" ".join)
dfGroupeReferents.loc[("spk1",slice(None)),:]
# dfGroupeReferents.loc[dfGroupeReferents.index.get_level_values(0).isin(["spk1","spk2"])]

Cat  \
speaker numChaine chaineIndex                                    
spk1    1.0       1.0                                  PRO:DEM   
                  2.0                                  PRO:DEM   
        2.0       1.0                                      NAM   
        4.0       1.0          DET:ART NOM PRP:det NOM PRP NOM   
        6.0       1.0                      DET:ART NOM PRP NAM   
...                                                        ...   
        1147.0    8.0              NOM PRP DET:ART NOM PRP NAM   
        1148.0    1.0                                      NAM   
        1149.0    1.0                      DET:ART NOM PRP NAM   
                  2.0                                  PRO:PER   
        1150.0    1.0                                      NAM   

                                                  Transcription  
speaker numChaine chaineIndex                                    
spk1    1.0       1.0                                      ceux  
                  2.0                                      ceux  
        2.0       1.0                                   Bernard  
        4.0       1.0           les champions du monde en titre  
        6.0       1.0                       l' équipe de France  
...                                                         ...  
        1147.0    8.0          capitaine de l' équipe de France  
        1148.0    1.0                                   Bernard  
        1149.0    1.0                       l' équipe de France  
                  2.0                                      nous  
        1150.0    1.0                                   Bernard  

[851 rows x 2 columns]

In [79]:
df.head()

,Transcription,Cat,Lemme,Referent,numChaine,chaineIndex,chaineDep,chaineApp,nPhrase,tour,type,tDeb,tFin,speaker,Mi-temps
0,bonsoir,INT,bonsoir,NaN,NaN,NaN,NaN,NaN,0,1-spk1,Colour commentary,00:00:36.628000,00:00:40.304000,spk1,1.0
1,",",PUN,",",NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,1.0
2,c',PRO:DEM,ce,NaN,NaN,NaN,NaN,NaN,0,1-spk1,Colour commentary,00:00:36.628000,00:00:40.304000,spk1,1.0
3,était,VER:impf,être,NaN,NaN,NaN,NaN,NaN,0,1-spk1,Colour commentary,00:00:36.628000,00:00:40.304000,spk1,1.0
4,ceux,PRO:DEM,celui,GTF,1.0,1.0,0.0,NaN,0,1-spk1,Colour commentary,00:00:36.628000,00:00:40.304000,spk1,1.0


In [80]:
dfSpkCat=df[(df.chaineIndex.notnull())].groupby(["speaker","Cat"])[["Referent"]].count()
dfSpkCat.reset_index(inplace=True)
dfSpkCat=dfSpkCat.pivot(columns = 'speaker', values = 'Referent',index="Cat").fillna(0)
dfSpkCat.columns=[c+"-%s"%match for c in dfSpkCat.columns]
dfSpkCat

,spk1-FrNz,spk1 spk2-FrNz,spk1 spk4-FrNz,spk1 spk5-FrNz,spk10-FrNz,spk2-FrNz,spk2 spk1-FrNz,spk4-FrNz,spk4 spk1-FrNz,spk4 spk2-FrNz,spk8-FrNz
Cat,,,,,,,,,,,
ADJ,32.0,0.0,0.0,1.0,0.0,31.0,2.0,2.0,0.0,2.0,0.0
ADV,9.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
DET:ART,90.0,6.0,0.0,0.0,0.0,47.0,0.0,5.0,0.0,1.0,0.0
DET:POS,8.0,1.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0
INT,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
KON,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
NAM,614.0,47.0,3.0,0.0,0.0,290.0,45.0,86.0,6.0,2.0,0.0
NOM,222.0,15.0,0.0,0.0,0.0,122.0,8.0,23.0,1.0,2.0,0.0
NUM,11.0,1.0,0.0,0.0,0.0,16.0,0.0,2.0,0.0,2.0,0.0


In [81]:
dfSpkCat.to_excel(writer, sheet_name="Spk-Dist-Cat-%s"%match)
# dfSpkCat.loc[dfSpkCat.index.get_level_values(0).isin(["spk1","spk2"])]

### Stats par période

In [82]:
def extractPer(df,mt=[1,2]):
    chainesPer={}
    perChaines={}
    for num in range(maxNumChaine):
        n=num+1
        dfChaine=df.loc[df.numChaine==n]
        # display(dfChaine)
        if len(dfChaine)>0 and dfChaine.iloc[0]["Mi-temps"] in mt:
            try:
                chainesPer[n]={"spk":dfChaine.speaker.values[0],
                               "ref":dfChaine.Referent.values[0],
                               "type":dfChaine.type.values[0],
                               "chaine":{}}
                indexMax=int(dfChaine.chaineIndex.max())
                # print(indexMax)
                if indexMax not in perChaines:
                    perChaines[indexMax]=0
                perChaines[indexMax]+=1
                for ind in range(indexMax):
                    i=ind+1
                    dfIndex=dfChaine.loc[dfChaine.chaineIndex==i]
                    # display(dfIndex)
                    chainesPer[n]["chaine"][i]=dfIndex[["Cat","Transcription"]].to_dict()
            except:
                print("pb",n,end=", ")
    return chainesPer,perChaines

#### longueur de chaines par période

In [83]:
def getPerLen(chainesPer):
    perLen={"Colour commentary":{n+1:0 for n in range(14)},"Play by play":{n+1:0 for n in range(14)}}
    for k,v in chainesPer.items():
        if v['type']==v['type']:
            per=v['type']
            lK=len(v["chaine"])
            if per not in perLen:
                perLen[per]={}
            if lK not in perLen[per]:
                perLen[per][lK]=0
            perLen[per][lK]+=1
    return perLen

In [84]:
chainesPerM1,perChainesM1=extractPer(df,mt=[1])
chainesPerM2,perChainesM2=extractPer(df,mt=[2])
perLenM1=getPerLen(chainesPerM1)
perLenM2=getPerLen(chainesPerM2)
perM1=pd.DataFrame.from_dict(perLenM1,orient="index").sort_index().T
perM2=pd.DataFrame.from_dict(perLenM2,orient="index").sort_index().T

In [85]:
dfPerLong=perM1.join(perM2,lsuffix="-MT1",rsuffix="-MT2")
dfPerLong["Colour commentary-Match"]=dfPerLong["Colour commentary-MT1"]+dfPerLong["Colour commentary-MT2"]
dfPerLong["Play by play-Match"]=dfPerLong["Play by play-MT1"]+dfPerLong["Play by play-MT2"]
dfPerLong.columns=[c+"-%s"%match for c in dfPerLong.columns]
dfPerLong

,Autre-MT1-FrNz,Colour commentary-MT1-FrNz,Play by play-MT1-FrNz,Autre-MT2-FrNz,Colour commentary-MT2-FrNz,Play by play-MT2-FrNz,Colour commentary-Match-FrNz,Play by play-Match-FrNz
1,30.0,194.0,129.0,27.0,216.0,88.0,410.0,217.0
2,10.0,65.0,37.0,7.0,71.0,31.0,136.0,68.0
3,5.0,22.0,17.0,NaN,22.0,20.0,44.0,37.0
4,2.0,11.0,11.0,3.0,15.0,7.0,26.0,18.0
5,NaN,8.0,5.0,1.0,9.0,2.0,17.0,7.0
6,1.0,4.0,2.0,1.0,7.0,3.0,11.0,5.0
7,NaN,4.0,2.0,NaN,4.0,4.0,8.0,6.0
8,NaN,3.0,2.0,NaN,5.0,0.0,8.0,2.0
9,NaN,0.0,1.0,NaN,2.0,0.0,2.0,1.0
10,NaN,4.0,0.0,NaN,2.0,1.0,6.0,1.0


In [86]:
dfPerLong.to_excel(writer, sheet_name="Type-Dist-Long-%s"%match)

In [87]:
# dfSpkCatDistBar.to_excel(writer, sheet_name="Type-Dist-Cat")

In [88]:
writer.close()